## Washington State Liquor and Cannabis Board (WSLCB) Open Data Portal

*For an introduction to the WSLCB, [see the README in the parent directory](../README.md).*

*For an introduction to the WSLCB's Socrata-based Open Data Portal, [see the README in this directory](./README.md).*

### Dataset: Licensed Businesses

* Canonical Dataset ID: **bhbp-x4eb**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/Licensing/Licensed-Businesses/u3zh-ri66
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/bhbp-x4eb

We'll be using the [`cannapy`](https://github.com/CannabisData/cannapy) library to access the portal data.  `cannapy` aims to provide an abstract interface for accessing and working with *Cannabis* data from around the world.  It utilizes [xmunoz](https://github.com/xmunoz)'s [`sodapy`](https://github.com/xmunoz/sodapy) client to access Socrata-based open data portals and can return data loaded into [Pandas DataFrames](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [11]:
import time
import cannapy.us.wa.wslcb.portal as wslcb
import pandas as pd

In [12]:
# Specify your own Socrata App Token if you plan to experiment
app_token = 'XaB9MBqc81C3KT4Vps6Wh5LZt'

# Instantiate a cannapy interface to the WSLCB open data portal
portal = wslcb.WSLCBPortal(app_token)

# We'll be using the Licensed Businesses dataset
dataset_id = 'bhbp-x4eb'

In [13]:
# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Mon Nov 13 12:35:56 2017


In [14]:
# Retrieve the dataset preloaded into a Pandas DataFrame
df = portal.get_dataframe(dataset_id)

# Validate we've got the right data by examining the first few rows
df.head()

,license,type,createdate,active,organization,address,address_line_2,city,state,zip,county,dayphone,ubi
0,362444,MARIJUANA RETAILER,20170105,ACTIVE (ISSUED),V.S.W.,13014 WA-99,NaN,EVERETT,WA,982040000,SNOHOMISH,2062715468,6035684740010001
1,421568,MARIJUANA RETAILER/MEDICAL MARIJUANA ENDORSEMENT,20171108,ACTIVE (ISSUED),BUD HUT,1717 NE ANDRESEN RD,NaN,VANCOUVER,WA,986616890,CLARK,4257896996,6035481120010003
2,412051,MARIJUANA PRODUCER TIER 3,20170905,ACTIVE (ISSUED),SOLSTICE,640 S SPOKANE ST,SUITE B,SEATTLE,WA,981342225,KING,4252838841,6031327110010003
3,412698,MARIJUANA PRODUCER TIER 1,20170817,ACTIVE (ISSUED),COOKIE CUTTER FARMS,23410 HIDDEN VALLEY RD STE A,NaN,GRANITE FALLS,WA,982523600,SNOHOMISH,2532312995,6040431000010001
4,417880,MARIJUANA RETAILER/MEDICAL MARIJUANA ENDORSEMENT,20161117,ACTIVE (ISSUED),LEGAL MARIJUANA SUPERSTORE,3610 BETHEL RD SE,SUITE 100,PORT ORCHARD,WA,983670000,KITSAP,3603401009,6033573230010003


In [15]:
# The DataFrame value_counts() histogramming method is useful for asking basic questions of any dataset
# column/Series with consistent text values.

# Question: How many licenses have been issued for each business type?
df.type.value_counts()

MARIJUANA PRODUCER TIER 2/MARIJUANA PROCESSOR       245
MARIJUANA RETAILER/MEDICAL MARIJUANA ENDORSEMENT    242
MARIJUANA PRODUCER TIER 3/MARIJUANA PROCESSOR       196
MARIJUANA PRODUCER TIER 1/MARIJUANA PROCESSOR       107
MARIJUANA PROCESSOR                                  94
MARIJUANA PRODUCER TIER 3                            38
MARIJUANA PRODUCER TIER 2                            30
MARIJUANA RETAILER                                   27
MARIJUANA PRODUCER TIER 1                            12
MARIJUANA TRANSPORTATION                              9
Name: type, dtype: int64

In [16]:
# Question: How many licenses are active vs. pending?
df.active.value_counts()

ACTIVE (ISSUED)     898
PENDING (ISSUED)    102
Name: active, dtype: int64

In [17]:
# Question: How many licenses are have been issued per county?
df.county.value_counts()

KING            117
SPOKANE         106
SNOHOMISH        98
OKANOGAN         64
THURSTON         57
WHATCOM          49
PIERCE           48
BENTON           40
GRANT            38
YAKIMA           35
SKAGIT           28
CLARK            28
CHELAN           25
MASON            25
GRAYS HARBOR     24
COWLITZ          21
STEVENS          21
ADAMS            18
PACIFIC          18
KITSAP           17
CLALLAM          16
KLICKITAT        13
DOUGLAS          12
WHITMAN          12
KITTITAS         11
JEFFERSON        11
ISLAND            9
LEWIS             8
PEND OREILLE      7
SKAMANIA          5
LINCOLN           5
WALLA WALLA       5
FERRY             2
WAHKIAKUM         2
SAN JUAN          2
ASOTIN            2
FRANKLIN          1
Name: county, dtype: int64

In [18]:
# The UBI column uniquely identifies each licensee, but obscures ownership of multiple licenses by the same entity.
# Let's break that column apart into its constituent parts:
# Unified Business Identifier (UBI): first nine digits
# Business ID Number: next three digits
# Location Number: last four digits
df_v2 = df.rename(columns={'ubi': 'ubi_source'})
df_v2['ubi'] = df_v2.ubi_source.str[0:9]
df_v2['ubi_business_id'] = df_v2.ubi_source.str[9:12]
df_v2['ubi_location'] = df_v2.ubi_source.str[12:]
df_v2.head()

,license,type,createdate,active,organization,address,address_line_2,city,state,zip,county,dayphone,ubi_source,ubi,ubi_business_id,ubi_location
0,362444,MARIJUANA RETAILER,20170105,ACTIVE (ISSUED),V.S.W.,13014 WA-99,NaN,EVERETT,WA,982040000,SNOHOMISH,2062715468,6035684740010001,603568474,001,0001
1,421568,MARIJUANA RETAILER/MEDICAL MARIJUANA ENDORSEMENT,20171108,ACTIVE (ISSUED),BUD HUT,1717 NE ANDRESEN RD,NaN,VANCOUVER,WA,986616890,CLARK,4257896996,6035481120010003,603548112,001,0003
2,412051,MARIJUANA PRODUCER TIER 3,20170905,ACTIVE (ISSUED),SOLSTICE,640 S SPOKANE ST,SUITE B,SEATTLE,WA,981342225,KING,4252838841,6031327110010003,603132711,001,0003
3,412698,MARIJUANA PRODUCER TIER 1,20170817,ACTIVE (ISSUED),COOKIE CUTTER FARMS,23410 HIDDEN VALLEY RD STE A,NaN,GRANITE FALLS,WA,982523600,SNOHOMISH,2532312995,6040431000010001,604043100,001,0001
4,417880,MARIJUANA RETAILER/MEDICAL MARIJUANA ENDORSEMENT,20161117,ACTIVE (ISSUED),LEGAL MARIJUANA SUPERSTORE,3610 BETHEL RD SE,SUITE 100,PORT ORCHARD,WA,983670000,KITSAP,3603401009,6033573230010003,603357323,001,0003


In [19]:
# Washington law/regulation states a maximum of three Producer/Processor or Retail licenses can be held
# per UBI.  We can validate that by asking: how many licenses are held by any particular UBI?
df_v2.ubi.value_counts().unique()

array([3, 2, 1])

In [20]:
# Question: How many and what kind of licenses are associated with each UBI?

# Calculate the number of licenses per UBI, and add the Series into a DataFrame for merging
licenses_per_ubi = pd.DataFrame(df_v2.groupby('ubi').size(), columns=['ubi_licenses'])

# Extract context fields from the source DataFrame, merge, and sort
ubi_and_organization = df_v2.loc[:, ['organization', 'type', 'ubi', 'ubi_business_id', 'ubi_location']]
licenses_per_org = pd.merge(ubi_and_organization, licenses_per_ubi, left_on='ubi', right_index=True)
licenses_per_org.sort_values(['ubi_licenses', 'ubi', 'ubi_business_id', 'ubi_location'], ascending=True, inplace=True)
licenses_per_org

,organization,type,ubi,ubi_business_id,ubi_location,ubi_licenses
811,CANNARAIL STATION,MARIJUANA RETAILER/MEDICAL MARIJUANA ENDORSEMENT,600128226,001,0001,1
545,GREEN HEAD CANNABIS,MARIJUANA RETAILER/MEDICAL MARIJUANA ENDORSEMENT,600314288,001,0001,1
657,OLIVER'S FARM,MARIJUANA PRODUCER TIER 3/MARIJUANA PROCESSOR,600369641,001,0002,1
174,VASHON ISLAND ORGANICS,MARIJUANA PRODUCER TIER 1/MARIJUANA PROCESSOR,600385770,001,0002,1
333,DUKE'S HILL CANNA,MARIJUANA PRODUCER TIER 2/MARIJUANA PROCESSOR,600435277,001,0003,1
406,ALASKAN GREEN ACRES,MARIJUANA PRODUCER TIER 1/MARIJUANA PROCESSOR,600449039,001,0002,1
834,JVARD,MARIJUANA PRODUCER TIER 2/MARIJUANA PROCESSOR,600497647,001,0002,1
643,APPLICATION SOFTWARE PRODUCTS,MARIJUANA PRODUCER TIER 3/MARIJUANA PROCESSOR,600505938,001,0003,1
27,CAREY INVESTMENTS,MARIJUANA PRODUCER TIER 2/MARIJUANA PROCESSOR,600519153,001,0002,1
684,CALVIN'S BEST,MARIJUANA PROCESSOR,600578588,001,0002,1
